In [36]:
import pandas as pd
from sklearn.model_selection import KFold
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

Read the CSV file

In [37]:
df = pd.read_csv("/Users/calvinli/Desktop/MS FIANL/StudentPerformanceFactors-1.csv")

In [38]:
df

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6602,25,69,High,Medium,No,7,76,Medium,Yes,1,High,Medium,Public,Positive,2,No,High School,Near,Female,68
6603,23,76,High,Medium,No,8,81,Medium,Yes,3,Low,High,Public,Positive,2,No,High School,Near,Female,69
6604,20,90,Medium,Low,Yes,6,65,Low,Yes,3,Low,Medium,Public,Negative,2,No,Postgraduate,Near,Female,68
6605,10,86,High,High,Yes,6,91,High,Yes,2,Low,Medium,Private,Positive,3,No,High School,Far,Female,68


In [39]:
df

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6602,25,69,High,Medium,No,7,76,Medium,Yes,1,High,Medium,Public,Positive,2,No,High School,Near,Female,68
6603,23,76,High,Medium,No,8,81,Medium,Yes,3,Low,High,Public,Positive,2,No,High School,Near,Female,69
6604,20,90,Medium,Low,Yes,6,65,Low,Yes,3,Low,Medium,Public,Negative,2,No,Postgraduate,Near,Female,68
6605,10,86,High,High,Yes,6,91,High,Yes,2,Low,Medium,Private,Positive,3,No,High School,Far,Female,68


選擇我們要使用的變數們

In [40]:
income_dummies = pd.get_dummies(df['Family_Income'], prefix='income', drop_first=True)
distance_dummies = pd.get_dummies(df['Distance_from_Home'], prefix='distance', drop_first=True)

In [41]:
df = pd.concat([df, income_dummies, distance_dummies], axis=1)
df.drop(['Family_Income', 'Distance_from_Home'], axis=1, inplace=True)


In [42]:
selected_cols = [
    'Hours_Studied',
    'Sleep_Hours',
    'Previous_Scores',
    'Attendance',
    'income_Low',  # dummy
    'income_Medium',    # dummy
    'distance_Moderate',  # dummy
    'distance_Near',       # dummy
    'Exam_Score'  # 目標變數
]

df = df[selected_cols].dropna()  # 移除缺漏值

In [43]:
df

,Hours_Studied,Sleep_Hours,Previous_Scores,Attendance,income_Low,income_Medium,distance_Moderate,distance_Near,Exam_Score
0,23,7,73,84,True,False,False,True,67
1,19,8,59,64,False,True,True,False,61
2,24,7,91,98,False,True,False,True,74
3,29,8,98,89,False,True,True,False,71
4,19,6,65,92,False,True,False,True,70
...,...,...,...,...,...,...,...,...,...
6602,25,7,76,69,False,False,False,True,68
6603,23,8,81,76,True,False,False,True,69
6604,20,6,65,90,True,False,False,True,68
6605,10,6,91,86,True,False,False,False,68


切分 5-Fold Cross Validation

In [44]:
kf = KFold(n_splits=5, shuffle=False)
folds = []

# 把 index 分成 5 折
for train_index, test_index in kf.split(df):
    train_df = df.iloc[train_index].copy()
    test_df = df.iloc[test_index].copy()
    folds.append((train_df, test_df))

In [45]:
train_df

,Hours_Studied,Sleep_Hours,Previous_Scores,Attendance,income_Low,income_Medium,distance_Moderate,distance_Near,Exam_Score
0,23,7,73,84,True,False,False,True,67
1,19,8,59,64,False,True,True,False,61
2,24,7,91,98,False,True,False,True,74
3,29,8,98,89,False,True,True,False,71
4,19,6,65,92,False,True,False,True,70
...,...,...,...,...,...,...,...,...,...
5281,24,5,94,61,False,True,True,False,64
5282,26,7,62,64,False,True,False,True,66
5283,24,5,64,71,False,True,False,True,66
5284,30,5,70,99,False,True,False,True,74


In [46]:
test_df

,Hours_Studied,Sleep_Hours,Previous_Scores,Attendance,income_Low,income_Medium,distance_Moderate,distance_Near,Exam_Score
5286,13,6,100,75,False,True,False,True,67
5287,28,4,98,96,True,False,False,False,73
5288,16,6,96,78,False,True,False,True,66
5289,25,6,54,73,False,True,True,False,67
5290,29,5,88,98,True,False,False,True,75
...,...,...,...,...,...,...,...,...,...
6602,25,7,76,69,False,False,False,True,68
6603,23,8,81,76,True,False,False,True,69
6604,20,6,65,90,True,False,False,True,68
6605,10,6,91,86,True,False,False,False,68


計算訓練資料中的節點位置

In [47]:
def compute_knots(train_df, feature_names, quantiles=[0.10, 0.25, 0.50, 0.75, 0.90]):
    knots_dict = {}
    for feature in feature_names:
        values = train_df[feature].values
        knots = np.quantile(values, quantiles)
        knots_dict[feature] = dict(zip(quantiles, knots))
    return knots_dict


In [48]:
# 要建立 spline 的數值變數（不要包含目標變數）
numeric_features = [
    'Hours_Studied',
    'Sleep_Hours',
    'Previous_Scores',
    'Attendance',
   
]

# 建立 spline knots
knots = compute_knots(train_df, numeric_features)

# 看看結果長怎樣
import pprint
pprint.pprint(knots)


{'Attendance': {0.1: np.float64(64.0),
                0.25: np.float64(70.0),
                0.5: np.float64(80.0),
                0.75: np.float64(90.0),
                0.9: np.float64(96.0)},
 'Hours_Studied': {0.1: np.float64(12.0),
                   0.25: np.float64(16.0),
                   0.5: np.float64(20.0),
                   0.75: np.float64(24.0),
                   0.9: np.float64(28.0)},
 'Previous_Scores': {0.1: np.float64(55.0),
                     0.25: np.float64(63.0),
                     0.5: np.float64(75.0),
                     0.75: np.float64(87.0),
                     0.9: np.float64(95.0)},
 'Sleep_Hours': {0.1: np.float64(5.0),
                 0.25: np.float64(6.0),
                 0.5: np.float64(7.0),
                 0.75: np.float64(8.0),
                 0.9: np.float64(9.0)}}


建立 Hinge 特徵

In [49]:
def add_hinge_features(df, knots_dict):
    df_hinge = df.copy()
    
    for feature, knot_levels in knots_dict.items():
        for q, knot_val in knot_levels.items():
            new_col = f"{feature}_hinge_{q}"
            df_hinge[new_col] = (df_hinge[feature] - knot_val).clip(lower=0)
            
    return df_hinge

In [50]:
train_df_hinge = add_hinge_features(train_df, knots)
test_df_hinge = add_hinge_features(test_df, knots)

In [51]:
print([col for col in train_df_hinge.columns if 'hinge' in col])


['Hours_Studied_hinge_0.1', 'Hours_Studied_hinge_0.25', 'Hours_Studied_hinge_0.5', 'Hours_Studied_hinge_0.75', 'Hours_Studied_hinge_0.9', 'Sleep_Hours_hinge_0.1', 'Sleep_Hours_hinge_0.25', 'Sleep_Hours_hinge_0.5', 'Sleep_Hours_hinge_0.75', 'Sleep_Hours_hinge_0.9', 'Previous_Scores_hinge_0.1', 'Previous_Scores_hinge_0.25', 'Previous_Scores_hinge_0.5', 'Previous_Scores_hinge_0.75', 'Previous_Scores_hinge_0.9', 'Attendance_hinge_0.1', 'Attendance_hinge_0.25', 'Attendance_hinge_0.5', 'Attendance_hinge_0.75', 'Attendance_hinge_0.9']


In [52]:
train_df_hinge


,Hours_Studied,Sleep_Hours,Previous_Scores,Attendance,income_Low,income_Medium,distance_Moderate,distance_Near,Exam_Score,Hours_Studied_hinge_0.1,...,Previous_Scores_hinge_0.1,Previous_Scores_hinge_0.25,Previous_Scores_hinge_0.5,Previous_Scores_hinge_0.75,Previous_Scores_hinge_0.9,Attendance_hinge_0.1,Attendance_hinge_0.25,Attendance_hinge_0.5,Attendance_hinge_0.75,Attendance_hinge_0.9
0,23,7,73,84,True,False,False,True,67,11.0,...,18.0,10.0,0.0,0.0,0.0,20.0,14.0,4.0,0.0,0.0
1,19,8,59,64,False,True,True,False,61,7.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,24,7,91,98,False,True,False,True,74,12.0,...,36.0,28.0,16.0,4.0,0.0,34.0,28.0,18.0,8.0,2.0
3,29,8,98,89,False,True,True,False,71,17.0,...,43.0,35.0,23.0,11.0,3.0,25.0,19.0,9.0,0.0,0.0
4,19,6,65,92,False,True,False,True,70,7.0,...,10.0,2.0,0.0,0.0,0.0,28.0,22.0,12.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5281,24,5,94,61,False,True,True,False,64,12.0,...,39.0,31.0,19.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
5282,26,7,62,64,False,True,False,True,66,14.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5283,24,5,64,71,False,True,False,True,66,12.0,...,9.0,1.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,0.0
5284,30,5,70,99,False,True,False,True,74,18.0,...,15.0,7.0,0.0,0.0,0.0,35.0,29.0,19.0,9.0,3.0


In [53]:
test_df_hinge

,Hours_Studied,Sleep_Hours,Previous_Scores,Attendance,income_Low,income_Medium,distance_Moderate,distance_Near,Exam_Score,Hours_Studied_hinge_0.1,...,Previous_Scores_hinge_0.1,Previous_Scores_hinge_0.25,Previous_Scores_hinge_0.5,Previous_Scores_hinge_0.75,Previous_Scores_hinge_0.9,Attendance_hinge_0.1,Attendance_hinge_0.25,Attendance_hinge_0.5,Attendance_hinge_0.75,Attendance_hinge_0.9
5286,13,6,100,75,False,True,False,True,67,1.0,...,45.0,37.0,25.0,13.0,5.0,11.0,5.0,0.0,0.0,0.0
5287,28,4,98,96,True,False,False,False,73,16.0,...,43.0,35.0,23.0,11.0,3.0,32.0,26.0,16.0,6.0,0.0
5288,16,6,96,78,False,True,False,True,66,4.0,...,41.0,33.0,21.0,9.0,1.0,14.0,8.0,0.0,0.0,0.0
5289,25,6,54,73,False,True,True,False,67,13.0,...,0.0,0.0,0.0,0.0,0.0,9.0,3.0,0.0,0.0,0.0
5290,29,5,88,98,True,False,False,True,75,17.0,...,33.0,25.0,13.0,1.0,0.0,34.0,28.0,18.0,8.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6602,25,7,76,69,False,False,False,True,68,13.0,...,21.0,13.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
6603,23,8,81,76,True,False,False,True,69,11.0,...,26.0,18.0,6.0,0.0,0.0,12.0,6.0,0.0,0.0,0.0
6604,20,6,65,90,True,False,False,True,68,8.0,...,10.0,2.0,0.0,0.0,0.0,26.0,20.0,10.0,0.0,0.0
6605,10,6,91,86,True,False,False,False,68,0.0,...,36.0,28.0,16.0,4.0,0.0,22.0,16.0,6.0,0.0,0.0


In [54]:
features = [
    'income_Medium', 'income_Low',
    'distance_Moderate', 'distance_Near',
    'Hours_Studied_hinge_0.1', 'Hours_Studied_hinge_0.25', 'Hours_Studied_hinge_0.5', 'Hours_Studied_hinge_0.75', 'Hours_Studied_hinge_0.9',
    'Sleep_Hours_hinge_0.1', 'Sleep_Hours_hinge_0.25', 'Sleep_Hours_hinge_0.5', 'Sleep_Hours_hinge_0.75', 'Sleep_Hours_hinge_0.9',
    'Previous_Scores_hinge_0.1', 'Previous_Scores_hinge_0.25', 'Previous_Scores_hinge_0.5', 'Previous_Scores_hinge_0.75', 'Previous_Scores_hinge_0.9',
    'Attendance_hinge_0.1', 'Attendance_hinge_0.25', 'Attendance_hinge_0.5', 'Attendance_hinge_0.75', 'Attendance_hinge_0.9'
]
target_col = 'Exam_Score'


In [55]:
features

['income_Medium',
 'income_Low',
 'distance_Moderate',
 'distance_Near',
 'Hours_Studied_hinge_0.1',
 'Hours_Studied_hinge_0.25',
 'Hours_Studied_hinge_0.5',
 'Hours_Studied_hinge_0.75',
 'Hours_Studied_hinge_0.9',
 'Sleep_Hours_hinge_0.1',
 'Sleep_Hours_hinge_0.25',
 'Sleep_Hours_hinge_0.5',
 'Sleep_Hours_hinge_0.75',
 'Sleep_Hours_hinge_0.9',
 'Previous_Scores_hinge_0.1',
 'Previous_Scores_hinge_0.25',
 'Previous_Scores_hinge_0.5',
 'Previous_Scores_hinge_0.75',
 'Previous_Scores_hinge_0.9',
 'Attendance_hinge_0.1',
 'Attendance_hinge_0.25',
 'Attendance_hinge_0.5',
 'Attendance_hinge_0.75',
 'Attendance_hinge_0.9']

In [56]:
def train_lp_spline(train_df, features, target_col='Exam_Score'):
    m = gp.Model("spline_regression")
    m.setParam("OutputFlag", 0)  # 不輸出中間 log

    n = train_df.shape[0]

    # 建立 β 變數（每個特徵一個），再加上截距 β0
    beta = {f: m.addVar(lb=-GRB.INFINITY, name=f'beta_{f}') for f in features}
    beta0 = m.addVar(lb=-GRB.INFINITY, name='beta0')

    # 每一筆資料的誤差 |ei|（要使用 e_pos 和 e_neg 表示絕對值）
    e_pos = []
    e_neg = []

    for i in range(n):
        e_pos.append(m.addVar(lb=0, name=f"e_pos_{i}"))
        e_neg.append(m.addVar(lb=0, name=f"e_neg_{i}"))

    # 建立 constraint: y = β0 + Σ(βj * xij) + e_neg - e_pos
    for i in range(n):
        row = train_df.iloc[i]
        xi_expr = gp.LinExpr()
        for f in features:
            xi_expr += beta[f] * row[f]
        m.addConstr(beta0 + xi_expr + e_neg[i] - e_pos[i] == row[target_col])

    # 目標函數：最小化 Σ (e_pos + e_neg)，即 MAE
    m.setObjective(gp.quicksum(e_pos[i] + e_neg[i] for i in range(n)), GRB.MINIMIZE)
    m.optimize()

    # 儲存解
    coefficients = {'beta0': beta0.X}
    for f in features:
        coefficients[f] = beta[f].X

    return coefficients


In [57]:
# 訓練
coef = train_lp_spline(train_df_hinge, features, target_col='Exam_Score')

# 預測
def predict_lp(df, coef, features):
    y_pred = []
    for _, row in df.iterrows():
        y_hat = coef['beta0']
        for f in features:
            y_hat += coef[f] * row[f]
        y_pred.append(y_hat)
    return y_pred

train_preds = predict_lp(train_df_hinge, coef, features)

# 評估 MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(train_df_hinge['Exam_Score'], train_preds)
print(f"Gurobi LP Train MAE: {mae:.4f}")


Gurobi LP Train MAE: 1.3797


In [58]:
def cross_validate_lp(df, features, target_col='Exam_Score', n_splits=5, random_state=42):
    kf = KFold(n_splits=n_splits, shuffle=False)
    fold_maes = []

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(df)):
        train_df_fold = df.iloc[train_idx].copy()
        test_df_fold = df.iloc[test_idx].copy()

        # 1. 使用 Gurobi 訓練模型
        coef_lp = train_lp_spline(train_df_fold, features, target_col)

        # 2. 預測測試集
        preds = predict_lp(test_df_fold, coef_lp, features)

        # 3. 計算 MAE
        mae = mean_absolute_error(test_df_fold[target_col], preds)
        fold_maes.append(mae)
        print(f"Fold {fold_idx + 1} MAE: {mae:.4f}")

    # 4. 整體 CV-MAE
    cv_mae = np.mean(fold_maes)
    print(f"\n✅ Overall CV-MAE: {cv_mae:.4f}")

    return fold_maes, cv_mae

In [59]:
fold_maes, overall_cv_mae = cross_validate_lp(train_df_hinge, features)


Fold 1 MAE: 1.4213
Fold 2 MAE: 1.3595
Fold 3 MAE: 1.4517
Fold 4 MAE: 1.3353
Fold 5 MAE: 1.3810

✅ Overall CV-MAE: 1.3897


原始函式的 加上 L1 正則化預算

In [60]:
def train_lp_spline_with_l1(train_df, features, target_col='Exam_Score', l1_budget=None):
    m = gp.Model("spline_lp_with_l1")
    m.setParam("OutputFlag", 0)  # 不輸出 Gurobi 日誌

    n = train_df.shape[0]

    # 建立 beta 和 beta_abs
    beta = {f: m.addVar(lb=-GRB.INFINITY, name=f'beta_{f}') for f in features}
    beta_abs = {f: m.addVar(lb=0, name=f'beta_abs_{f}') for f in features}

    # 截距項
    beta0 = m.addVar(lb=-GRB.INFINITY, name='beta0')

    # 加入絕對值限制：beta_abs >= |beta|
    for f in features:
        m.addConstr(beta_abs[f] >= beta[f], name=f"abs_pos_{f}")
        m.addConstr(beta_abs[f] >= -beta[f], name=f"abs_neg_{f}")

    # 殘差項：e_pos, e_neg
    e_pos = []
    e_neg = []
    for i in range(n):
        e_pos.append(m.addVar(lb=0, name=f"e_pos_{i}"))
        e_neg.append(m.addVar(lb=0, name=f"e_neg_{i}"))

    # 每筆資料的 constraint
    for i in range(n):
        row = train_df.iloc[i]
        xi_expr = gp.LinExpr()
        for f in features:
            xi_expr += beta[f] * row[f]
        m.addConstr(beta0 + xi_expr + e_neg[i] - e_pos[i] == row[target_col], name=f"data_point_{i}")

    # 拆出 regular 與 hinge features
    hinge_features = [f for f in features if 'hinge' in f]
    regular_features = [f for f in features if f not in hinge_features]
    beta_abs_regular = [beta_abs[f] for f in regular_features]
    beta_abs_hinge = [beta_abs[f] for f in hinge_features]

    # 加入 L1 預算約束
    if l1_budget is not None:
        m.addConstr(
            gp.quicksum(beta_abs_regular) + gp.quicksum(beta_abs_hinge) <= l1_budget,
            name="l1_budget"
        )

    # 最小化總殘差
    m.setObjective(gp.quicksum(e_pos) + gp.quicksum(e_neg), GRB.MINIMIZE)
    m.optimize()

    # 儲存係數
    coefficients = {'beta0': beta0.X}
    for f in features:
        coefficients[f] = beta[f].X

    return coefficients


In [61]:
coeffs = train_lp_spline_with_l1(
    train_df=train_df_hinge,
    features=features,  # 你的 spline + dummy 變數清單
    target_col='Exam_Score',
    l1_budget=5  # 控制預算（你可以試不同的值）
)

嘗試不同的 L1 預算值，使用 K-fold Cross Validation 計算每個預算值下的平均 MAE，最後找出最佳的預算

In [62]:
def cross_validate_l1_budgets(train_df, features, target_col='Exam_Score', budgets=range(2, 11), n_splits=5):
    results = {}
    kf = KFold(n_splits=n_splits, shuffle=False)

    for budget in budgets:
        fold_maes = []
        for train_idx, val_idx in kf.split(train_df):
            train_fold = train_df.iloc[train_idx]
            val_fold = train_df.iloc[val_idx]

            try:
                coeffs = train_lp_spline_with_l1(train_fold, features, target_col, l1_budget=budget)
            except:
                continue  # 忽略解失敗的 case

            val_pred = coeffs['beta0'] + sum(coeffs[f] * val_fold[f] for f in features)
            mae = mean_absolute_error(val_fold[target_col], val_pred)
            fold_maes.append(mae)

        if fold_maes:
            mean_mae = np.mean(fold_maes)
            results[budget] = {
                'mae_list': fold_maes,
                'mean_mae': mean_mae
            }
            print(f"L1 Budget = {budget}, CV MAE = {mean_mae:.4f}")
        else:
            print(f"L1 Budget = {budget}, No valid folds.")

    return results


In [63]:
results = cross_validate_l1_budgets(
    train_df=train_df_hinge,
    features=features,
    target_col='Exam_Score',
    budgets=range(2, 6),
    n_splits=5
)


L1 Budget = 2, CV MAE = 1.4011
L1 Budget = 3, CV MAE = 1.3906
L1 Budget = 4, CV MAE = 1.3879
L1 Budget = 5, CV MAE = 1.3897


In [64]:
best_budget = min(results.items(), key=lambda x: x[1]['mean_mae'])[0]
print(f"\nBest L1 Budget: {best_budget} with MAE = {results[best_budget]['mean_mae']:.4f}")



Best L1 Budget: 4 with MAE = 1.3879


In [65]:
# 2. 計算「無限制」的 CV MAE（baseline）
unregularized_maes = []
kf = KFold(n_splits=5, shuffle=False)

for train_idx, val_idx in kf.split(train_df_hinge):
    train_fold = train_df_hinge.iloc[train_idx]
    val_fold = train_df_hinge.iloc[val_idx]

    coeffs = train_lp_spline_with_l1(train_fold, features, target_col='Exam_Score', l1_budget=None)

    val_pred = coeffs['beta0'] + sum(coeffs[f] * val_fold[f] for f in features)
    mae = mean_absolute_error(val_fold['Exam_Score'], val_pred)
    unregularized_maes.append(mae)

unregularized_cv_mae = np.mean(unregularized_maes)
print(f"No L1 Constraint (Unregularized) CV MAE = {unregularized_cv_mae:.4f}")


No L1 Constraint (Unregularized) CV MAE = 1.3897


Refit 模型

In [66]:
# 重新訓練模型（使用整個訓練集 & 最佳 L1 Budget）
final_coeffs = train_lp_spline_with_l1(train_df_hinge, features, target_col='Exam_Score', l1_budget=best_budget)



In [67]:
# 篩選出非截距且係數絕對值大於 0.001 的變數
nonzero_coeffs = {k: v for k, v in final_coeffs.items() if k != 'beta0' and abs(v) > 0.001}

# 按照效用（絕對值）排序，從大到小
sorted_coeffs = sorted(nonzero_coeffs.items(), key=lambda x: abs(x[1]), reverse=True)

# 印出排序後的結果
print("所有非截距且係數絕對值大於 0.001 的參數（按效用排序）：")
for k, v in sorted_coeffs:
    print(f"{k}: {v:.4f}")


所有非截距且係數絕對值大於 0.001 的參數（按效用排序）：
income_Low: -1.0251
distance_Near: 0.8467
income_Medium: -0.4533
Hours_Studied_hinge_0.1: 0.4329
distance_Moderate: 0.3539
Attendance_hinge_0.1: 0.2719
Hours_Studied_hinge_0.25: -0.1501
Previous_Scores_hinge_0.9: -0.1182
Attendance_hinge_0.25: -0.0784
Sleep_Hours_hinge_0.75: -0.0730
Attendance_hinge_0.9: 0.0625
Previous_Scores_hinge_0.1: 0.0527
Hours_Studied_hinge_0.75: 0.0336
Previous_Scores_hinge_0.75: 0.0243
Attendance_hinge_0.75: -0.0091
Hours_Studied_hinge_0.9: 0.0068
Previous_Scores_hinge_0.5: -0.0051
Previous_Scores_hinge_0.25: -0.0019


In [68]:
# 將所有係數（包含 beta0）放入列表中，並依照絕對值大小排序
sorted_all_coeffs = sorted(final_coeffs.items(), key=lambda x: abs(x[1]), reverse=True)

# 印出所有係數（包含截距）
print("所有係數（含截距，按絕對值排序）：")
for k, v in sorted_all_coeffs:
    print(f"{k}: {v:.4f}")


所有係數（含截距，按絕對值排序）：
beta0: 59.6763
income_Low: -1.0251
distance_Near: 0.8467
income_Medium: -0.4533
Hours_Studied_hinge_0.1: 0.4329
distance_Moderate: 0.3539
Attendance_hinge_0.1: 0.2719
Hours_Studied_hinge_0.25: -0.1501
Previous_Scores_hinge_0.9: -0.1182
Attendance_hinge_0.25: -0.0784
Sleep_Hours_hinge_0.75: -0.0730
Attendance_hinge_0.9: 0.0625
Previous_Scores_hinge_0.1: 0.0527
Hours_Studied_hinge_0.75: 0.0336
Previous_Scores_hinge_0.75: 0.0243
Attendance_hinge_0.75: -0.0091
Hours_Studied_hinge_0.9: 0.0068
Previous_Scores_hinge_0.5: -0.0051
Previous_Scores_hinge_0.25: -0.0019
Attendance_hinge_0.5: -0.0006
Hours_Studied_hinge_0.5: 0.0000
Sleep_Hours_hinge_0.1: 0.0000
Sleep_Hours_hinge_0.25: 0.0000
Sleep_Hours_hinge_0.5: 0.0000
Sleep_Hours_hinge_0.9: 0.0000


Part3:

In [76]:
from sklearn.metrics import mean_absolute_error
from scipy.stats import laplace


def run_prediction_interval_construction(train_df, features, target_col='Exam_Score', best_budget=5, n_splits=5, S=100):
    kf = KFold(n_splits=n_splits, shuffle=False)

    all_coverages = []
    all_msis = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(train_df)):
        print(f"\nFold {fold + 1} ----------------------------")
        train_fold = train_df.iloc[train_idx]
        val_fold = train_df.iloc[val_idx]

        # Step 1: 訓練主模型
        coeffs = train_lp_spline_with_l1(train_fold, features, target_col, l1_budget=best_budget)

        # 預測值
        val_pred = coeffs['beta0'] + sum(coeffs[f] * val_fold[f] for f in features)
        val_pred = val_pred.values if hasattr(val_pred, 'values') else val_pred  # 保持一致格式

        # Step 1-1: 殘差 r_i
        residuals = val_fold[target_col].values - val_pred
        log_sq_residuals = np.log(residuals ** 2 + 1e-8)  # 加 epsilon 防止 log(0)

        # Step 1-2: 對 log(residual^2) 做回歸來估計 sigma^2(x)
        X_reg = train_fold[features].values
        y_reg = np.log((train_fold[target_col].values - (
            coeffs['beta0'] + sum(coeffs[f] * train_fold[f] for f in features)).values) ** 2 + 1e-8)
        reg = LinearRegression().fit(X_reg, y_reg)
        log_sigma2_pred = reg.predict(val_fold[features].values)
        sigma2_pred = np.exp(log_sigma2_pred)
        bi = np.sqrt(sigma2_pred / 2)  # Laplace 參數

        # Step 2: Laplace Simulation for each test point
        y_true = val_fold[target_col].values
        y_hat = val_pred

        lower_bounds = []
        upper_bounds = []

        for i in range(len(y_hat)):
            e_samples = laplace.rvs(loc=0, scale=bi[i], size=S)
            y_samples = y_hat[i] + e_samples
            L = np.percentile(y_samples, 5)
            U = np.percentile(y_samples, 95)
            lower_bounds.append(L)
            upper_bounds.append(U)

        # Step 3: 評估 Coverage 與 MSIS
        yk_bar = np.mean(y_true)
        coverage = np.mean([(y_true[i] >= lower_bounds[i]) and (y_true[i] <= upper_bounds[i]) for i in range(len(y_true))])

        msis_scores = []
        alpha = 0.1
        for i in range(len(y_true)):
            Ui = upper_bounds[i]
            Li = lower_bounds[i]
            yi = y_true[i]
            msis_i = (Ui - Li) / yk_bar \
                + 2 / alpha * max(0, Li - yi) / yk_bar \
                + 2 / alpha * max(0, yi - Ui) / yk_bar
            msis_scores.append(msis_i)

        fold_msis = np.mean(msis_scores)
        all_coverages.append(coverage)
        all_msis.append(fold_msis)

        print(f"Coverage Rate: {coverage:.3f}, MSIS: {fold_msis:.3f}")

    print("\n=== Final Evaluation Results ===")
    print(f"Average Coverage Rate: {np.mean(all_coverages):.3f}")
    print(f"Average MSIS: {np.mean(all_msis):.3f}")

In [70]:
run_prediction_interval_construction(
    train_df=train_df_hinge,
    features=features,
    target_col='Exam_Score',
    best_budget=best_budget,
    n_splits=5
)


Fold 1 ----------------------------
Coverage Rate: 0.563, MSIS: 0.205

Fold 2 ----------------------------
Coverage Rate: 0.570, MSIS: 0.186

Fold 3 ----------------------------
Coverage Rate: 0.557, MSIS: 0.210

Fold 4 ----------------------------
Coverage Rate: 0.583, MSIS: 0.182

Fold 5 ----------------------------
Coverage Rate: 0.574, MSIS: 0.194

=== Final Evaluation Results ===
Average Coverage Rate: 0.570
Average MSIS: 0.195


家明 新加上～

In [80]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import numpy as np

def run_prediction_interval_construction(train_df, features, target_col, best_budget, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=False)

    coverage_rates = []
    msis_list = []

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(train_df)):
        train_fold = train_df.iloc[train_idx].copy()
        val_fold = train_df.iloc[val_idx].copy()

        # 模型訓練
        coeffs = train_lp_spline_with_l1(train_fold, features, target_col, l1_budget=best_budget)

        # 預測值（驗證集）
        val_preds = predict_lp(val_fold, coeffs, features)

        # 預測值（訓練集，用來計算殘差 std）
        train_preds = predict_lp(train_fold, coeffs, features)
        residuals = train_fold[target_col].values - np.array(train_preds)
        std_residual = np.std(residuals)

        # 建立預測區間
        lower = val_preds - 1.96 * std_residual
        upper = val_preds + 1.96 * std_residual

        # Coverage rate
        y_true = val_fold[target_col].values
        covered = (y_true >= lower) & (y_true <= upper)
        coverage_rate = np.mean(covered)

        # MSIS（這邊簡化為區間寬度的平均）
        msis = np.mean(upper - lower)

        # 記錄
        coverage_rates.append(coverage_rate)
        msis_list.append(msis)

        print(f"Fold {fold_idx+1} ----------------------------")
        print(f"Coverage Rate: {coverage_rate:.3f}, MSIS: {msis:.3f}")

    # === 最終輸出：平均值 ===
    avg_coverage = np.mean(coverage_rates)
    avg_msis = np.mean(msis_list)

    print("\n=== Final Evaluation Results ===")
    print(f"Average Coverage Rate: {avg_coverage:.3f}")
    print(f"Average MSIS: {avg_msis:.3f}")


In [78]:
run_prediction_interval_construction(
    train_df=train_df_hinge,
    features=features,
    target_col='Exam_Score',
    best_budget=best_budget,
    n_splits=5
)

Fold 1 ----------------------------
Coverage Rate: 0.995, MSIS: 10.481
Fold 2 ----------------------------
Coverage Rate: 0.995, MSIS: 10.577
Fold 3 ----------------------------
Coverage Rate: 0.993, MSIS: 10.513
Fold 4 ----------------------------
Coverage Rate: 0.987, MSIS: 9.806
Fold 5 ----------------------------
Coverage Rate: 0.983, MSIS: 9.103

=== Final Evaluation Results ===
Average Coverage Rate: 0.991
Average MSIS: 10.096


用Laplace-Based Interval Simulation

In [86]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

def laplace_prediction_interval_once(train_df, features, target_col, best_budget, n_splits=5, alpha=0.05):
    kf = KFold(n_splits=n_splits, shuffle=False)

    coverage_rates = []
    interval_widths = []

    z = np.log(2 / alpha)  # Laplace multiplier, 約為 2.97（對應 95% CI）

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(train_df)):
        train_fold = train_df.iloc[train_idx].copy()
        val_fold = train_df.iloc[val_idx].copy()

        # 模型訓練
        coeffs = train_lp_spline_with_l1(train_fold, features, target_col, l1_budget=best_budget)

        # 預測值
        val_preds = predict_lp(val_fold, coeffs, features)
        train_preds = predict_lp(train_fold, coeffs, features)

        # 使用訓練集殘差計算 Laplace scale b
        residuals = train_fold[target_col].values - np.array(train_preds)
        mae = np.mean(np.abs(residuals))
        b = mae / np.sqrt(2)

        # 預測區間
        lower = val_preds - z * b
        upper = val_preds + z * b

        y_true = val_fold[target_col].values
        covered = (y_true >= lower) & (y_true <= upper)

        # 評估
        coverage = np.mean(covered)
        width = np.mean(upper - lower)

        coverage_rates.append(coverage)
        interval_widths.append(width)

        print(f"Fold {fold_idx+1} - Coverage Rate: {coverage:.3f}, Interval Width: {width:.3f}")

    # 平均結果
    avg_coverage = np.mean(coverage_rates)
    avg_width = np.mean(interval_widths)

    print("\n=== Final Evaluation Results (Single Run, Laplace-Based) ===")
    print(f"Average Coverage Rate: {avg_coverage:.3f}")
    print(f"Average Interval Width (MSIS): {avg_width:.3f}")


In [87]:
laplace_prediction_interval_once(
    train_df=train_df_hinge,
    features=features,
    target_col='Exam_Score',
    best_budget=best_budget,
    n_splits=5,
    alpha=0.05  # 95% 預測區間
)


Fold 1 - Coverage Rate: 0.973, Interval Width: 7.156
Fold 2 - Coverage Rate: 0.979, Interval Width: 7.238
Fold 3 - Coverage Rate: 0.973, Interval Width: 7.117
Fold 4 - Coverage Rate: 0.966, Interval Width: 7.272
Fold 5 - Coverage Rate: 0.974, Interval Width: 7.205

=== Final Evaluation Results (Single Run, Laplace-Based) ===
Average Coverage Rate: 0.973
Average Interval Width (MSIS): 7.198
